In [1]:
%load_ext literary.module

# Notebook Loader

In [1]:
from pathlib import Path

from ..transpile.exporter import LiteraryExporter

In [2]:
import linecache
from importlib.machinery import SourcelessFileLoader

import nbformat

In [3]:
class NotebookLoader(SourcelessFileLoader):
    """Sourceless Jupyter Notebook loader"""

    def __init__(self, fullname: str, path: str, exporter):
        super().__init__(fullname, path)

        self._exporter = exporter

    def _update_linecache(self, path: str, source: str):
        linecache.cache[path] = (
            len(source),
            None,
            source.splitlines(keepends=True),
            path,
        )

    def get_code(self, fullname: str):
        path = self.get_filename(fullname)
        body = self.get_transpiled_source(path)
        # Ensure that generated source is available for tracebacks
        self._update_linecache(path, body)
        return compile(body, path, "exec")

    def get_transpiled_source(self, path: str):
        nb = nbformat.read(path, as_version=nbformat.NO_CONVERT)
        body, resources = self._exporter.from_notebook_node(nb)
        return body

Let's do something **meta**. Let's try importing this notebook source.

First we're going to need an exporter

In [4]:
exporter = LiteraryExporter()

In [5]:
file_path = Path.cwd() / "loader.ipynb"
loader = NotebookLoader("loader.ipynb", file_path, exporter)

Is this a notebook?

In [6]:
loader.get_data(file_path.name)[:100]

b'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "id": "b8c9dad6-af40-42b8-a071-2e6ce66c0d9e",\n   "m'

Yep! The loader doesn't expose `get_source`, because that wouldn't make sense for a JSON backed notebook. But, let's see if we can get the transpiled source from `linecache`. We need to generate the code object to trigger the code-generation.

In [7]:
code = loader.get_code(file_path.name)

Now we can inspect the generated lines from linecache.

In [8]:
source = "".join(linecache.getlines(file_path))
print(source)

"""# Notebook Loader
"""
import linecache
from importlib.machinery import SourcelessFileLoader
import nbformat

class NotebookLoader(SourcelessFileLoader):
    """Sourceless Jupyter Notebook loader"""

    def __init__(self, fullname: str, path: str, exporter):
        super().__init__(fullname, path)
        self._exporter = exporter

    def _update_linecache(self, path: str, source: str):
        linecache.cache[path] = (len(source), None, source.splitlines(keepends=True), path)

    def get_code(self, fullname: str):
        path = self.get_filename(fullname)
        body = self.get_transpiled_source(path)
        self._update_linecache(path, body)
        return compile(body, path, 'exec')

    def get_transpiled_source(self, path: str):
        nb = nbformat.read(path, as_version=nbformat.NO_CONVERT)
        (body, resources) = self._exporter.from_notebook_node(nb)
        return body


Wow! That's ... kinda messed up. Let's just be sure that we can compile that. We can take the generated `code` object and execute it in a local namespace:

In [9]:
generated_ns = {}
exec(code, generated_ns)

And ...

In [10]:
assert "NotebookLoader" in generated_ns